<a href="https://colab.research.google.com/github/RashmitVartak/DAV/blob/main/DAV_EXP9_63.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment - 9: Create a World Map with d3.js

Name:Rashmit Vartak

Div: D11AD

Rollno:63



**Title of Experiment**:Create a World Map with d3.js

**Outcome of Experiment :**
Implement visualization techniques to given data sets using Python.

**Theory :**

D3.js can bind any arbitrary data to a Document Object Model (DOM), and then, through the use of JavaScript, CSS, HTML and SVG, apply transformations to the document that are driven by that data. The result can be simple HTML output, or interactive SVG charts with dynamic behavior like animations, transitions, and interaction. All the data transformations and renderings are done client-side, in the browser.

D3.js can be used for much more than just DOM manipulation, or to draw charts. D3.js is extremely powerful when it comes to handling geographical information. Manipulating and presenting geographic data can be very tricky, but building a map with a D3.js is quite simple.

We will draw a world map based on the data stored in a JSON-compatible data format. You just need to define the size of the map and the geographic projection to use (more about that later), define an SVG element, append it to the DOM, and load the map data using JSON. Map styling is done via CSS.

GeoJSON is “a format for encoding a variety of geographic data structures”. It is designed to represent discrete geometry objects grouped into feature collections of name/value pairs.

TopoJSON is an extension of GeoJSON, which can encode topology where geometries are “stitched together from shared line segments called arcs”. TopoJSON eliminates redundancy by storing relational information between geographic features, not merely spatial information. As a result, geometry is much more compact and combined where geometries share features. This results with 80% smaller typical TopoJSON file than its GeoJSON equivalent.

In [5]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Interactive World Map with D3.js</title>
    <style>
        #map {
            width: 100%;
            height: 600px;
        }

        .country {
            fill: #ccc;
            stroke: #fff;
            stroke-width: 0.5;
        }

        .country:hover {
            fill: #999;
        }
    </style>
</head>
<body>
    <div id="map"></div>

    <script src="https://d3js.org/d3.v5.min.js"></script>
    <script src="https://d3js.org/topojson.v3.min.js"></script>
    <script>
        const width = 960;
        const height = 600;

        const svg = d3.select("#map")
            .append("svg")
            .attr("width", width)
            .attr("height", height)
            .append("g");

        const projection = d3.geoMercator()
            .scale(130)
            .translate([width / 2, height / 1.5]);

        const path = d3.geoPath().projection(projection);

        const colorScale = d3.scaleOrdinal(d3.schemeCategory10);

        d3.json("https://raw.githubusercontent.com/d3/d3.github.com/master/world-110m.v1.json").then(function(world) {
            svg.selectAll("path")
                .data(topojson.feature(world, world.objects.countries).features)
                .enter().append("path")
                .attr("class", "country")
                .attr("d", path)
                .style("fill", (d, i) => colorScale(i)) // Assigning colors based on index
                .on("click", clicked);
        });

        function clicked(d) {
            const bounds = path.bounds(d);
            const dx = bounds[1][0] - bounds[0][0];
            const dy = bounds[1][1] - bounds[0][1];
            const x = (bounds[0][0] + bounds[1][0]) / 2;
            const y = (bounds[0][1] + bounds[1][1]) / 2;
            const scale = Math.max(1, Math.min(8, 0.9 / Math.max(dx / width, dy / height)));
            const translate = [width / 2 - scale * x, height / 2 - scale * y];

            svg.transition()
                .duration(750)
                .call(zoom.transform, d3.zoomIdentity.translate(translate[0], translate[1]).scale(scale));
        }

        const zoom = d3.zoom()
            .scaleExtent([1, 8])
            .on("zoom", zoomed);

        svg.call(zoom);

        function zoomed() {
            svg.selectAll("path")
                .attr("transform", d3.event.transform);
        }

        // Add event listener to the document to detect clicks outside of the map
        document.addEventListener('click', function(event) {
            if (!event.target.closest('#map')) {
                // If click occurs outside of the map, reset zoom to original view
                svg.transition()
                    .duration(750)
                    .call(zoom.transform, d3.zoomIdentity);
            }
        });
    </script>
</body>
</html>
